In [41]:
import requests
from bs4 import BeautifulSoup
import re

In [2]:
req = requests.get('https://www.imsdb.com/scripts/Matrix,-The.html')
soup = BeautifulSoup(req.content, 'lxml')

In [5]:
script = soup.find('pre')

In [157]:
script.text[:1000]

"\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\t\t\t\t\tTHE MATRIX\n\n\n\n\t\t\t\t\tWritten by\n\n\t\t\t\tLarry and Andy Wachowski\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\t\t\t\t\tApril 8, 1996\n\n\n\n\n\n\n\n\tFADE IN ON:\n\n\tCOMPUTER SCREEN\n\n\tSo close it has no boundaries.\n\n\tA blinking cursor pulses in the electric darkness like a\n\theart coursing with phosphorous light, burning beneath\n\tthe derma of black-neon glass.\n\n\tA PHONE begins to RING, we hear it as though we were \n\tmaking the call.  The cursor continues to throb,\n\trelentlessly patient, until --\n\n\t\t\t\t\tMAN (V.O.)\n\t\t\tHello?\n\n\tData now slashes across the screen, information flashing\n\tfaster than we read.\n\n\t\t\t\t\tSCREEN\n\t\t\tCall trans opt:  received.\n\t\t\t2-19-96  13:24:18  REC:Log>\n\n\t\t\t\t\tWOMAN (V.O.)\n\t\t\tI'm inside.  Anything to report?\n\n\tWe listen to the phone conversation as though we were on\n\ta third line.  The man's name is CYPHER.  The woman, \n\tTRINITY.\n\n\t\

names are preceded by 2 \n\n\t\t\t\t\t, dialogue is preceded by \n\t\t\t, stage direction is preceded by \n\n\t.  Problems that immediately come to mind: 
- making sure preceding spoken line is dialogue by someone else and not some transition
- making sure the format is the same between all scripts

With the neat formatting, that's all I can think of.

Leap of faith: conditions listed are going to apply to other scripts.  Unlikely, but let's see how it goes.

In [156]:
possible_characters = set([re.sub('[\n\t]', '', tagged.text) for tagged in script.findAll('b') if 'INT.' not in tagged.text and 'EXT.' not in tagged.text and ':' not in tagged.text])

In [114]:
all_lines = [chunk.split('\n\t\t\t') for chunk in script.text.split('\n\n\t\t\t\t\t')]

In [122]:
all_lines[101]

['NEO',
 'Shitshitshit.\n\n\n\tEXT.  SKYSCRAPER\n\n\tThe downtown office of CorTechs, a software development\n\tcompany.\n\n\n\tINT.  CORTECHS OFFICE\n\n\tThe main offices are along each wall, the windows\n\toverlooking downtown Chicago.\n\n\tRHINEHEART, the ultimate company man, lectures Neo\n\twithout looking at him, typing at his computer\n\tcontinuously.\n\n\tNeo stares at two window cleaners on a scaffolding\n\toutside, dragging their rubber squeegees down across the\n\tsurface of the glass.']

In [141]:
void_directions = lambda line: re.sub('\n\n+\t.*$', '', line, flags=re.S)
response = [(lines[0], ' '.join([void_directions(line) for line in lines[1:]])) for lines in all_lines if lines[0] == 'NEO']

In [137]:
neo_lines_index = [index - 1 for index, lines in enumerate(all_lines) if lines[0]=='NEO']

In [138]:
len(neo_lines_index)

175

In [142]:
prompts = [(lines[0], ' '.join([void_directions(line) for line in lines[1:]])) for index, lines in enumerate(all_lines) if index in neo_lines_index]

In [143]:
[zip(prompts, response)]

[(('SCREEN',
   "JACKON:  I heard Morpheus has been on this board. SUPERASTIC:  Morpheus doesn't even exist and the Matrix is nothing  but an advertising gimmick 4 a new game. TIMAXE:  All I want to know is Trinity really a girl? LODIII:  87% of all women on line are really men. QUARK:  The Matrix is a euphemism  for the government. SUPERASTIC:  No, The Matrix is the system controlling our lives. TIMAXE:  You mean MTV. SUPERASTIC:  I mean Sega. FOS4:  ALL HAIL SEGA!!!"),
  ('NEO', "Fuckin' idiots don't know shit.")),
 (('SCREEN',
   "SUPERASTIC:  Who said that? JACKON:  Who's Neo? GIBSON:  This is a private board.\n If you want to know, follow the white rabbit."),
  ('NEO', 'What the hell...')),
 (('VOICE (O.S.)', 'Hey, Tommy-boy!  You in there?'),
  ('NEO', 'What do you want, Anthony?')),
 (('ANTHONY',
   'I need your help, man.  Desperate. They got me, man.  The shackles of  fascism.'),
  ('NEO', 'You got the money this time?')),
 (('DUJOUR', "Why don't you come to the party with us?